In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [3]:
library(Metrics)
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource)

In [4]:
partition <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=1)

trainset <- df.training[partition,] %>% select(-SalePrice)

testset <- df.training[-partition,] %>% select(-sale_price_log)

setdiff(trainset %>% colnames, testset %>% colnames)
setdiff(testset %>% colnames, trainset %>% colnames)

[1] "sale_price_log"

[1] "SalePrice"

In [5]:
tran.res <- kaggle.house$trans$doItAll(trainset, testset %>% select(-SalePrice))
trainset.ready <- tran.res$df.training
testset.ready <- tran.res$df.testing

# Should be only numeric columns
stopifnot(0 == trainset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)
stopifnot(0 == testset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

c(trainset.ready %>% nrow, testset.ready %>% nrow)

c('1. ', setdiff(trainset.ready %>% colnames, testset.ready %>% colnames))
c('2. ', setdiff(testset.ready %>% colnames, trainset.ready %>% colnames))

[1] 731 729

[1] "1. "            "sale_price_log"

[1] "2. "

In [6]:
# sanity check

testset %>% group_by(BldgType) %>% summarise(median(log(SalePrice)))

cbind(testset.ready %>% select(BldgType.new), testset %>% select(BldgType)) %>% 
    group_by(BldgType) %>% summarise(min(BldgType.new), max(BldgType.new))


BldgType,median(log(SalePrice))
1Fam,12.03112
2fmCon,11.72597
Duplex,11.75587
Twnhs,11.73318
TwnhsE,12.02719


BldgType,min(BldgType.new),max(BldgType.new)
1Fam,12.02874,12.02874
2fmCon,11.81967,11.81967
Duplex,11.82704,11.82704
Twnhs,11.90834,11.90834
TwnhsE,12.06681,12.06681


In [14]:
model.lm <- lm(sale_price_log ~ ., data=trainset.ready)

y_predicted <- predict(model.lm, testset.ready) %>% as.vector
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)

paste0("Testset RMSE of sale_price_log: ", rmse(y_predicted, y_actual))

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)

paste0("Testset SalePrice mean prediction error in %: ", mean(100 * abs(sp_actual - sp_predicted) / sp_actual))

glance(model.lm)
tidy(model.lm) %>% arrange(p.value) %>% filter(p.value < 0.1)

Warning message in predict.lm(model.lm, testset.ready):
“prediction from a rank-deficient fit may be misleading”

[1] "Testset RMSE of sale_price_log: 0.164441396534423"

[1] "Testset SalePrice mean prediction error in %: 10.9175977723496"

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.9198736,0.9105622,0.1183822,98.79078,9.916579e-310,77,563.2732,-970.5464,-612.1822,9.165379,654


term,estimate,std.error,statistic,p.value
has_pool,-3.915731e+00,4.461804e-01,-8.776115,1.466269e-17
PoolArea,6.295255e-03,7.462382e-04,8.435986,2.104228e-16
OverallQual,5.372890e-02,6.740840e-03,7.970654,7.043781e-15
X2ndFlrSF,1.993465e-04,2.587912e-05,7.702988,4.939619e-14
X1stFlrSF,2.666292e-04,3.507409e-05,7.601886,1.016644e-13
OverallCond,4.046574e-02,5.699261e-03,7.100173,3.257721e-12
Neighborhood.new,1.982228e-01,2.952735e-02,6.713195,4.134913e-11
Condition1.new,2.402280e-01,4.573909e-02,5.252138,2.036153e-07
GarageCars,6.607362e-02,1.483146e-02,4.454963,9.869009e-06
MSZoning.new,1.688297e-01,3.964042e-02,4.259028,2.354441e-05


In [30]:
sample.data <- df.training %>% sample_n(round(0.75 * nrow(df.training)))
sample.data %>% nrow
N = 3
partitions <- caret::createDataPartition(y=sample.data$sale_price_log, p=.6, list=F, times=N) %>% 
                as.data.frame %>% as.list

partitions[[1]] %>% length

[1] 1095

[1] 659

In [162]:
N = 3

partitions <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=N) %>% 
                as.data.frame %>% as.list

buildModel <- function (partition) { 
    trainset <- df.training[partition,] %>% select(-SalePrice)
    testset <- df.training[-partition,] %>% select(-sale_price_log)

    tran.res <- kaggle.house$trans$doItAll(trainset, testset %>% select(-SalePrice))
    trainset.ready <- tran.res$df.training
    testset.ready <- tran.res$df.testing

    model <- lm(sale_price_log ~ ., data=trainset.ready)
    
    y_predicted <- predict(model, testset.ready) %>% as.vector
    y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)
    
    list(model=model, rmse=rmse(y_predicted, y_actual))
}

fits <- partitions %>% as.data.frame %>% as.list %>% map(buildModel)

Warning message in predict.lm(model, testset.ready):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(model, testset.ready):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(model, testset.ready):
“prediction from a rank-deficient fit may be misleading”

In [9]:
rmse.sample <- fits %>% map(function(item) { item$rmse }) %>% unlist %>% as.vector
rmse.sample

[1] 0.1464964 0.1467018 0.1634096 0.1329504 0.1715179 0.1334099 0.1664902
 [8] 0.2651426 0.1443541 0.1420386 0.1942162 0.1404823 0.1508373 0.1413518
[15] 0.1343987 0.1560970 0.4567993 0.1543027 0.1292011 0.1804074

In [10]:
r.squared.sample <- fits %>% map(function(item) { glance(item$model)[['r.squared']] }) %>% unlist %>% as.vector
r.squared.sample

[1] 0.9213545 0.9225239 0.9316627 0.9032638 0.9466337 0.9107395 0.9318271
 [8] 0.9134051 0.9197099 0.9124321 0.9184909 0.9042219 0.9213573 0.9091171
[15] 0.9061400 0.9217063 0.9146403 0.9250254 0.8988045 0.9031479

In [36]:
coeff.sample <- fits %>% map(function(item) { tidy(item$model) }) %>% bind_rows
coeff.sample %>% 
    arrange(term, estimate) %>% 
    group_by(term) %>% 
    mutate(n=n()) %>% 
    filter(p.value < 0.00000001, n == N) %>%
    distinct(term) %>%
    `[[`(1) %>% paste(collapse = '+')

[1] "BsmtFinSF1+Condition2.new+has_pool+MSZoning.new+Neighborhood.new+OverallCond+OverallQual+PoolArea+X1stFlrSF+X2ndFlrSF"

In [160]:
df.tr <- 
    trainset.ready %>% 
    mutate(
        GrLivArea.log = log(GrLivArea),
        X1stFlrSF.log = log(X1stFlrSF),
        X2ndFlrSF.log = log(X2ndFlrSF + 1),
        LotArea.log = log(LotArea)
    )

df.te <- 
    testset.ready %>% 
    mutate(
        GrLivArea.log = log(GrLivArea),
        X1stFlrSF.log = log(X1stFlrSF),
        X2ndFlrSF.log = log(X2ndFlrSF + 1),
        LotArea.log = log(LotArea)
    )

model.lm <- lm(
    sale_price_log ~ OverallQual+Neighborhood.new+GrLivArea.log+X1stFlrSF.log+BsmtFullBath+
                     LotArea.log+has_central_air+YearRemodAdd+GarageCars, 
    data=df.tr)


glance(model.lm)
tidy(model.lm)

y_predicted <- predict(model.lm, df.te) %>% as.vector
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)
rmse(y_predicted, y_actual)

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)
mean(100 * abs(sp_actual - sp_predicted) / sp_actual)

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.8668367,0.8651745,0.1483862,521.4895,1.039939e-308,10,362.4922,-702.9843,-652.4458,15.87532,721


term,estimate,std.error,statistic,p.value
(Intercept),0.51335179,0.6846550006,0.7497963,4.536220e-01
OverallQual,0.08052613,0.0069156870,11.6439812,7.804972e-29
Neighborhood.new,0.27587930,0.0272502951,10.1239013,1.292317e-22
GrLivArea.log,0.31553726,0.0235967954,13.3720388,1.365706e-36
X1stFlrSF.log,0.09243856,0.0232050584,3.9835522,7.478674e-05
BsmtFullBath,0.06390370,0.0113162861,5.6470563,2.348617e-08
LotArea.log,0.08222360,0.0128285174,6.4094389,2.638188e-10
has_central_air,0.16017446,0.0221639189,7.2268112,1.262187e-12
YearRemodAdd,0.00187511,0.0003446257,5.4410047,7.264625e-08
GarageCars,0.06376398,0.0100036715,6.3740579,3.284150e-10


[1] 0.141228

[1] 10.5194

In [ ]:
N = 20

partitions <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=N) %>% 
                as.data.frame %>% as.list


fits <- partitions %>% as.data.frame %>% as.list %>% map(buildModel)